In [36]:
import torch.nn as nn
import torch, math
import time
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [37]:
batch_size = 64

In [38]:
df = pd.read_csv("../python-docker/Swedbank_A/x_Swedbank_A_transformer.csv", sep=";")

In [39]:
df["target"] = df["close"].shift(-1)
df = df[:-1]

In [40]:
df.tail()

,open,close,rsi,macd,sin_month,cos_month,sin_day,cos_day,sin_hour,cos_hour,sin_min,cos_min,target
55031,150.24,150.22,79.8246,0.0030,0.866025,0.5,0.207912,0.978148,0.895163,0.445738,0.000000,1.000000,150.38
55032,150.24,150.38,76.4706,0.0129,0.866025,0.5,0.207912,0.978148,0.895163,0.445738,0.104528,0.994522,150.38
55033,150.36,150.38,76.4706,0.0112,0.866025,0.5,0.207912,0.978148,0.895163,0.445738,0.207912,0.978148,150.58
55034,150.40,150.58,76.4706,0.0161,0.866025,0.5,0.207912,0.978148,0.895163,0.445738,0.309017,0.951057,150.80
55035,150.58,150.80,76.4706,0.0134,0.866025,0.5,0.207912,0.978148,0.895163,0.445738,0.406737,0.913545,150.88


In [41]:
times = sorted(df.index.values)  # get the times
last_10pct = sorted(df.index.values)[-int(0.1*len(times))]  # get the last 10% of the times
last_20pct = sorted(df.index.values)[-int(0.2*len(times))]  # get the last 20% of the times

test_data = df[(df.index >= last_10pct)].values
val_data = df[(df.index >= last_20pct) & (df.index < last_10pct)].values
train_data = df[(df.index < last_20pct)].values

In [43]:
def build_input_output(window_size, data):
    X = []
    y = []
    for i in range(window_size, len(data)-1):
        X.append(data[i-window_size:i][:,:-1])
        y.append([data[i-window_size:i][:,-1]])
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [44]:
x_train, y_train = build_input_output(10, train_data)
x_val, y_val = build_input_output(10, val_data)
x_test, y_test = build_input_output(10, test_data)

In [45]:
print(x_train.shape)

torch.Size([44018, 10, 12])


In [46]:
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)

train_data_loader = DataLoader(train_dataset, batch_size=batch_size)
dev_data_loader = DataLoader(val_dataset, batch_size=batch_size)
test_data_loader = DataLoader(test_dataset, batch_size=1)

In [54]:
class Transformer(nn.Module):
    # d_model : number of features
    def __init__(self,feature_size=7,num_layers=3,dropout=0):
        super(Transformer, self).__init__()

        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=feature_size, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, device):
        mask = self._generate_square_subsequent_mask(len(src)).to(device)
        print(mask)
        output = self.transformer_encoder(src,mask)
        output = self.decoder(output)
        return output
        



In [55]:
def train_model(model, train_data_loader, dev_data_loader, loss_fn, optimizer, epochrange, batchsize, device):
    for epoch in range(epochrange):
        losses = []
        n_correct = 0
        model.train()
        for x, y in train_data_loader:
            optimizer.zero_grad()
            y = y.type(dtype).permute(2,0,1)
            x = x.type(dtype).transpose(0,1)
            
            print(x.shape)
            pred = model(x, device)
            loss = loss_fn(pred, y)
            
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
            

        # Compute accuracy and loss in the entire training set
        train_avg_loss = sum(losses)/len(losses)    
        
        dev_avg_loss, preds = evaluate_model(dev_data_loader, model, loss_fn, device)
        print(preds[0])
        
        # Display metrics
        display_str = 'Epoch {} '
        display_str += '\tLoss: {:.3f} '
        display_str += '\tLoss (val): {:.3f}'
        print(display_str.format(epoch, train_avg_loss, dev_avg_loss))

In [56]:
def evaluate_model(data, model, loss_fn, device):
    losses = []
    predictions = []
    model.eval()
    with torch.no_grad():
        for x, y in data:
            y = y.type(dtype).permute(2,0,1)
            x = x.type(dtype).transpose(0,1)
            pred = model(x, device)
            loss = loss_fn(pred, y)
            losses.append(loss.item())
            predictions.extend(pred.tolist())
        avg_loss = sum(losses)/len(losses)    
    
    return avg_loss, predictions

In [57]:
device = torch.device('cuda:0')
dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU
model = Transformer(feature_size=5).to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_model(model, train_data_loader, dev_data_loader, loss_fn, optimizer, 10, 2, device)

torch.Size([10, 64, 12])
tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0')


AssertionError: 

In [21]:
_, preds = evaluate_model(test_data_loader, model, loss_fn, device)

In [24]:
preds

[[[150.8126983642578]],
 [[150.8126983642578]],
 [[150.81271362304688]],
 [[150.8126983642578]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.8126983642578]],
 [[150.8126983642578]],
 [[150.8126983642578]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.8126983642578]],
 [[150.8126983642578]],
 [[150.8126983642578]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81271362304688]],
 [[150.81

In [25]:
#y_data_ = list(zip(*test_data))
#items_plot = [y_data_borpi[1][t] for t in range(len(y_data_borpi[1]))]
#plt.plot(list(range(len(preds))), preds)
#plt.plot(list(range(len(items_plot))), items_plot)
#axes = plt.gca()
#axes.set_ylim([145,170])
#axes.set_xlim([320000,325000])
#plt.show()